In [22]:
#IMPORTING SETUP PACKAGES
import statsmodels.formula.api as smf
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)

In [23]:
number_of_top_items = '5000'
plot_name = number_of_top_items[0] + 'k'

#print(plot_name)

df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top'+number_of_top_items+'_seller_YTD_group_by_item_visit_20210226.pickle')


In [24]:
print(df.item_code.nunique())



4488


In [25]:
df.head()

,min_date,max_date,item_code,item_price,delivery_weeks,avg_sales,avg_unique_views,conv_rate,bin,item_parent_item_code,item_main_category,item_sub_category_1,item_sub_category_2,log_price,log_delivery_weeks,log_sales,log_avg_unique_views,log_conv_rate
0,2020-01-01,2020-05-23,000000001000012530,99.99,1,0.520833,35.743056,0.014573,144,000000008000000774,ESSZIMMERSTÜHLE&BÄNKE,BÄNKE,BÄNKE,4.605071,0.000100,-0.652133,3.576359,-4.221842
1,2020-01-01,2020-01-02,000000001000012794,219.99,2,0.000000,27.000000,0.000001,2,000000008000000826,BETTEN,BETTGESTELLE,MASSIVHOLZBETTEN,5.393583,0.693197,-9.210340,3.295841,-9.210340
2,2020-01-01,2020-03-13,000000001000021922,129.99,2,1.589041,147.191781,0.010797,73,000000001000021922-P,BAD,BADSCHRÄNKE,WASCHBECKENUNTERSCHRÄNKE,4.867458,0.693197,0.463194,4.991737,-4.519385
3,2020-01-01,2020-01-01,000000001000022179,549.99,3,0.000000,50.000000,0.000001,1,000000008000001920,BETTEN,BETTGESTELLE,POLSTERBETTEN,6.309900,1.098646,-9.210340,3.912025,-9.210340
4,2020-01-01,2020-01-01,000000001000024320,99.99,1,0.000000,8.000000,0.000001,1,000000008000002133,MATRATZEN&LATTENROSTE,MATRATZEN,KALTSCHAUMMATRATZEN,4.605071,0.000100,-9.210340,2.079454,-9.210340


In [26]:
print(df.dtypes)

min_date                  object
max_date                  object
item_code                 object
item_price               float64
delivery_weeks             int64
avg_sales                float64
avg_unique_views         float64
conv_rate                float64
bin                        int32
item_parent_item_code     object
item_main_category        object
item_sub_category_1       object
item_sub_category_2       object
log_price                float64
log_delivery_weeks       float64
log_sales                float64
log_avg_unique_views     float64
log_conv_rate            float64
dtype: object


In [27]:
def filter_df(DF = df,bins=6, avg_PDP = 19.):
    df_filt = DF[DF['bin'] > bins]
    df_items = df_filt.groupby('item_code').item_price.nunique() > 1
    df_filt  = df_filt.merge(df_items,on='item_code',how='inner')
    df_filt  = df_filt[df_filt['item_price_y']== True]
    #filter PDP
    #print(df_filt)
    df_filt = df_filt[df_filt['avg_unique_views'] > avg_PDP]
    #print(df_filt)
    return df_filt

In [28]:
df_filt = filter_df(df)

df_filt.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top5k_seller_YTD_filter_price_1w_20210226.pickle')

In [29]:
#def weighted_median(df, val, weight):
#    df_sorted = df.sort_values(val)
#    cumsum = df_sorted[weight].cumsum()
##    cutoff = df_sorted[weight].sum() / 2.
#   return df_sorted[cumsum >= cutoff][val].iloc[0]

In [30]:
#df_filt['median'] = weighted_median(df_filt,'item_price_x','bin')

In [31]:
#df_filt.groupby(['item_code']).median()

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('read_tables').getOrCreate()

from pyspark.sql import Window
import pyspark.sql.functions as F

df_spark = spark.createDataFrame(df_filt)



In [37]:
grp_window = Window.partitionBy('item_code')
magic_percentile = F.expr('percentile_approx(item_price_x, 0.5)')

df_spark= df_spark.withColumn('med_val', magic_percentile.over(grp_window))



In [38]:
df_spark.show()

+----------+----------+------------------+------------+--------------+-------------------+------------------+--------------------+---+---------------------+--------------------+-------------------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+------------+-------+
|  min_date|  max_date|         item_code|item_price_x|delivery_weeks|          avg_sales|  avg_unique_views|           conv_rate|bin|item_parent_item_code|  item_main_category|item_sub_category_1| item_sub_category_2|         log_price|  log_delivery_weeks|           log_sales|log_avg_unique_views|      log_conv_rate|item_price_y|med_val|
+----------+----------+------------------+------------+--------------+-------------------+------------------+--------------------+---+---------------------+--------------------+-------------------+--------------------+------------------+--------------------+--------------------+--------------------+----------------